In [ ]:
!pip install kafka-python pyspark==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install confluent-kafka


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from confluent_kafka import Producer


In [ ]:
pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Credential to confluent-cloud
bootstrap_servers = 'pkc-6ojv2.us-west4.gcp.confluent.cloud:9092'
security_protocol = 'SASL_SSL'
sasl_mechanism = 'PLAIN'
sasl_plain_username = '7LSJTV6ECYBKIDMH'
sasl_plain_password = '6jLYdOnP+vH8msoYrWfsZfbxdrtWgsrod1nLVZGYJqHkMxQO49r4+ifTVnOW4A5M'

In [ ]:
from pyspark.sql import SparkSession

scala_version = '2.12'  # TODO: Ensure this is correct
spark_version = '3.2.1'
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.2.0'
]
spark = SparkSession.builder\
   .master("local")\
   .appName("kafka-example")\
   .config("spark.jars.packages", ",".join(packages))\
   .getOrCreate()
spark

In [ ]:
from kafka import KafkaProducer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
import re
import zipfile


In [ ]:

spark = SparkSession.builder.appName('RequestClustering').getOrCreate()


# most optimal feature for clustering
feature_columns = ['host', 'time', 'status_code', 'data_size']

# Assemble the features into a feature vector
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')


In [ ]:
import numpy as np
# Define the clustering algorithm
k = 5 # most optiomal number of clusters
kmeans = KMeans(featuresCol='features', k=k)

# Function to calculate similarity between features and cluster centers
def calculate_similarity_func(features, cluster_centers):
    distances = [np.linalg.norm(features - center) for center in cluster_centers]
    return min(distances)

# Define a UDF to calculate similarity between a request and the cluster center
calculate_similarity = udf(lambda features: calculate_similarity_func(features, cluster_centers), DoubleType())



In [ ]:
# Load data from the log file into a DataFrame, use regular expressions to clean data
log_file = './42MBSmallServerLog.log'
log_pattern = r'(\S+) - - \[(.*?)\] "(.*?)" (\d+) (\d+)'
log_data = []

with open(log_file, 'r') as file:
    for line in file:
        match = re.match(log_pattern, line)
        if match:
            log_data.append((match.group(1), match.group(2), match.group(4), match.group(5)))

requests_df = spark.createDataFrame(log_data, schema='host STRING, time STRING, status_code STRING, data_size STRING')

In [ ]:
requests_df.show()

+---------------+--------------------+-----------+---------+
|           host|                time|status_code|data_size|
+---------------+--------------------+-----------+---------+
|    .236.105.42|26/Dec/2118:12:00...|        500|    13359|
| 220.69.190.164|26/Dec/2118:12:00...|        304|    34734|
| 33.208.243.254|26/Dec/2118:12:00...|        500|    27982|
|  177.70.88.105|26/Dec/2118:12:00...|        500|    24106|
|  10.237.41.225|26/Dec/2118:12:00...|        200|    26324|
| 134.56.148.101|26/Dec/2118:12:00...|        403|     1376|
|  103.57.36.200|26/Dec/2118:12:00...|        500|     5130|
|    15.177.4.33|26/Dec/2118:12:00...|        200|    32155|
|188.157.130.200|26/Dec/2118:12:00...|        403|    57425|
|    1.59.51.230|26/Dec/2118:12:00...|        303|    20113|
| 199.192.22.229|26/Dec/2118:12:00...|        200|     5927|
|  95.77.174.217|26/Dec/2118:12:00...|        404|     9489|
| 213.218.154.74|26/Dec/2118:12:00...|        403|    39723|
|  141.78.203.63|26/Dec/

In [ ]:

kafka_bootstrap_servers = bootstrap_servers
kafka_topic = 'logs'
producer_config = {
    'bootstrap.servers': kafka_bootstrap_servers,
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': sasl_plain_username,
    'sasl.password': sasl_plain_password
}

producer = Producer(producer_config)

batch_size = 1000  # Number of requests to batch together
batch = []  # List to store batched requests

for row in requests_df.collect():
    request = ','.join([str(elem) for elem in row])
    batch.append(request.encode('utf-8'))
    
    if len(batch) >= batch_size:
        # Serialize the batched requests into a single bytes-like object
        serialized_batch = b'\n'.join(batch)
        
        # Send the serialized batched requests in a single produce call
        producer.produce(kafka_topic, value=serialized_batch)
        producer.flush()
        batch = []  

# Send any remaining requests in the batch
if batch:
    serialized_batch = b'\n'.join(batch)
    producer.produce(kafka_topic, value=serialized_batch)
    producer.flush()

producer.close()


In [ ]:
# Perform clustering on the requests DataFrame
feature_vector = assembler.transform(requests_df)
model = kmeans.fit(feature_vector)
cluster_centers = model.clusterCenters()

In [ ]:
#Cluster assign
requests_df = requests_df.withColumn('cluster', calculate_similarity('features'))
spark.stop()